In [2]:
import calendar
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import dash_table as dt
import  psycopg2
from psycopg2 import Error
from sqlalchemy import text
from datetime import date
from datetime import timedelta

In [3]:
sql_text1 = """with agents as (select u.id agent_id,u.created_at::date as date_debut,u.username, concat(u.first_name,' ',u.last_name) as agent,r.name as agent_role,u.role_id,zone_id,z.name as zone,id_tenant from users u
                    inner join(select * from zones)z on u.zone_id=z.id
                    inner join(select * from roles)r on r.id = u.role_id
                    where u.is_active is True and blocked is False and u.role_id in(11,18,5,14,4))
                    select a.*,superviseur,sup_username,sup_id from agents a
                    inner join(select agent as superviseur,username as sup_username,agent_id as sup_id, zone_id as sup_zone_id from agents where role_id=4)aa on a.zone_id = aa.sup_zone_id
                    where id_tenant !=3 and a.role_id!=4"""
sql_text2 = """select s.name as stock_name,stock_type,us.user_id as agent_id ,pt.name as product_type,pt.category,
                   count(p.serial)::int as stock
            from users_stocks us,stocks s,stocks_products sp,products p,product_types pt
            where s.id = us.stock_id and sp.stock_id = s.id and p.id = sp.product_id
            and pt.id = p.product_type_id and s.deleted is False
            group by 1,2,3,4,5"""
sql_text3 = """ select a.created_at::date,to_char(a.created_at,'yyyy')::int as annee, to_char(a.created_at,'mm')::int as mois,
                   to_char(a.created_at,'dd')::int as jour,u.id as agent_id,u.zone_id,concat(u.first_name,' ',u.last_name) as agent,
                   u.id_tenant,g.name as group_prix,case when a.arrhes isnull then a.total_amount else a.arrhes end as deposit ,count(a.slug) as ventes
            from users u
            inner join (select * from accounts) a on a.user_id = u.id
            inner join (select * from groups) g on g.id = a.group_id
            where u.is_active is True and blocked is False and u.role_id in(11,18,5,14) and u.id_tenant!=3
            and u.id !=6 and a.status !=6
            group by 1,2,3,4,5,6,7,8,9,10 """
sql_text4 = """ select u.id agent_id,u.username sup_username, concat(u.first_name,' ',u.last_name) as superviseur,z.name as zone,id_tenant from users u
            inner join(select * from zones)z on u.zone_id=z.id
            inner join(select * from roles)r on r.id = u.role_id
            where u.is_active is True and blocked is False and u.role_id=4 """
sql_text5 = """select agent_id,closed_date,agent,type_ticket, to_char(closed_date,'yyyy')::int as annee,
            to_char(closed_date,'mm')::int mois,to_char(closed_date,'dd')::int jour,status,ticket
            from
            (select u.id agent_id, case when status = 'cloturer' and t.closed_at is not null then  t.closed_at::date
                                when status = 'cloturer' and t.closed_at isnull then  t.solved_at::date
                                when status = 'customer_paid' and t.paid_at is not null then t.paid_at::date
                                when status = 'customer_paid' and t.paid_at isnull then t.solved_at::date
                                when status = 'à faire'  then t.created_at::date end as closed_date, 
            concat(u.first_name, ' ',u.last_name) agent,t.type_ticket, t.status,count(t.id) as ticket
        from tickets t left join users u on t.assigned_to = u.id
        where t.solved_at is not null and assigned_at is not null
        and t.type_ticket in ('Ticket technique', 'Reposséder')
        and t.assigned_to is not null and u.role_id in (11,18,5,14)
        and status in ('cloturer','customer_paid') and deleted is False
             group by 1,2,3,4,5)tab
        """
sql_text6 = """select u.id as agent_id,  concat(u.first_name, ' ',u.last_name) agent,
                    t.type_ticket, count(t.id) as backlog
                from tickets t
                    left join users u on t.assigned_to = u.id
                where status = 'à faire' and assigned_at is not null
                and t.type_ticket in ('Ticket technique', 'Reposséder')
                and t.assigned_to is not null and u.role_id in (11,18,5,14) and t.deleted is False
                group by 1,2,3"""

# Connection to oolusolar base
try:
    connection = psycopg2.connect(user='chartio_read_only_user',
                                  password='2ZVF01USUWTKV3K9JJFY',
                                  host='oolu-main-postgresql.cfa4plgxjs0u.eu-central-1.rds.amazonaws.com',
                                  port='5432',
                                  database='oolusolar_analytics')
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print('You are Successfully connected to - ', record, '\n')
except (Exception, Error) as error:
    print(" Connection failed, try again", error)
    cursor.close()

df_ag_sup = pd.read_sql_query(sql_text1, connection)
df_stock = pd.read_sql_query(sql_text2, connection)
df_vente = pd.read_sql_query(sql_text3, connection)
df_sup = pd.read_sql_query(sql_text4, connection)
df_ticket = pd.read_sql_query(sql_text5, connection)
df_backlog = pd.read_sql_query(sql_text6, connection)

cursor.close()
connection.close()
print('data have been updated')


You are Successfully connected to -  ('PostgreSQL 9.5.24 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

data have been updated


In [4]:
df_ag_sup.head()

,agent_id,date_debut,username,agent,agent_role,role_id,zone_id,zone,id_tenant,superviseur,sup_username,sup_id
0,2617,2019-10-21,kedougoupoly09,Mamadou Kaba Diaby,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
1,2929,2020-01-14,guindo,Seydou GUINDO,Agent Commercial,14,16,Zone houndé,4,Emmanuel KABRE,emmanuel.kabre,153
2,2618,2019-10-21,kedougoupoly10,Boye Sara Diallo,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
3,1303,2018-11-27,agentcom82,Mbaye Dia,Agent Commercial,14,30,LOUGA,1,THIERNO FAYE,thierno.faye@oolusolar.com,1561
4,1180,2018-02-01,niono02,Adama Diabate,Agent Commercial,14,11,SEGOU,2,Dougoutigui Diarra,dougoutigui@oolusolar.com,3488
